In [1]:
import pandas as pd
from datetime import datetime
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
power_measurements_Header = [];

power_measurements = []
power_stats = []
fork_exit_exec_data = []

current = 0;

#Replace this with the dump file's path
with open('/content/drive/MyDrive/HSBC/march-to-sustainability/output.txt') as file:
    
    for line in file:

      line = line.split()

      if (len(line) > 0 and line[0]=='Time'):
        power_measurements_Header = line
        current = 1
        continue

      elif (len(line) > 0 and line[0]=='--------'):
        current = 2
        continue

      elif (len(line) > 0 and line[0]=='Log'):
        current = 3
        continue

      if (current == 1):
        power_measurements.append(line)

      elif (current == 2):
        if (line[0] == "Summary:"):
          current = 0
          continue
        power_stats.append(line)
      
      elif (current == 3):
        fork_exit_exec_data.append(line)



In [4]:
df_power_measurements = pd.DataFrame(power_measurements, columns = power_measurements_Header)
df_power_measurements

,Time,User,Nice,Sys,Idle,IO,Run,Ctxt/s,IRQ/s,Fork,Exec,Exit,Watts
0,12:29:55,0.2,0.3,0.1,98.8,0.6,1,335,124,1,0,1,8.97
1,12:30:05,0.1,0.0,0.1,98.8,1.1,1,301,111,0,0,9,8.91
2,12:30:15,0.1,0.0,0.2,98.1,1.6,1,298,112,3,0,2,8.97
3,12:30:25,0.2,0.0,0.1,98.9,0.8,1,301,105,2,0,4,8.93
4,12:30:35,0.1,0.0,0.1,99.1,0.7,1,292,106,2,0,0,8.93
5,12:30:45,0.2,0.0,0.1,97.8,1.9,1,333,116,2,0,2,8.92
6,12:30:55,0.1,0.0,0.1,99.1,0.6,1,319,122,1,0,1,8.98
7,12:31:05,0.2,0.0,0.1,99.3,0.5,1,346,115,1,0,1,8.95
8,12:31:15,0.2,0.0,0.1,99.2,0.6,1,332,113,2,0,4,8.89
9,12:31:25,0.1,0.0,0.1,99.2,0.5,1,283,103,3,0,2,8.89


Change the path to whatever you want it to be in your google drive - this will output the basic measurements shown above

In [5]:
basic_power_measurements_path = '/content/drive/MyDrive/HSBC/march-to-sustainability/basic_power_measurements.csv'

with open(basic_power_measurements_path, 'w', encoding = 'utf-8-sig') as f:
  df_power_measurements.to_csv(f)

#DON'T RERUN THIS BLOCK AFTER FIRST TIME

In [6]:
timeStamps = df_power_measurements["Time"].tolist()
wattages = df_power_measurements["Watts"].tolist()

current_timestamp_index = 0;
timeStamp_to_process_dict = {}

timeStamp_to_process_dict[timeStamps[current_timestamp_index]] = []

for item in fork_exit_exec_data:

  #check if current item pertains to an unknown process, if yes we can skip the iteration
  if (item[len(item)-1] == '(<unknown>)'):
    continue

  #timestamp for item
  item_time = datetime.strptime(item[0], '%H:%M:%S')
  #timestamp from main table
  key_time_stamp = datetime.strptime(timeStamps[current_timestamp_index], '%H:%M:%S')

  if item_time <= key_time_stamp:
    #insert into corresponding dictionary entry
    timeStamp_to_process_dict[timeStamps[current_timestamp_index]].append(item)
  
  else:
    #assign random wattages 
    num_timestamps = len(timeStamp_to_process_dict[timeStamps[current_timestamp_index]])
    weightageList = random.sample(range(1, 100), num_timestamps)
    weightSum = sum(weightageList)
    total_watts = wattages[current_timestamp_index]

    for i in range(num_timestamps):
      watts = float(total_watts)*weightageList[i]/weightSum
      #changed - toInsert = str(watts) + " Watts"
      toInsert = watts
      timeStamp_to_process_dict[timeStamps[current_timestamp_index]][i].insert(1, toInsert)

    #increment timestamp index and assign empty array to value
    current_timestamp_index += 1
    timeStamp_to_process_dict[timeStamps[current_timestamp_index]] = []

    if item_time <= key_time_stamp:
      #insert into corresponding dictionary entry
      timeStamp_to_process_dict[timeStamps[current_timestamp_index]].append(item)

#assign random wattages 
num_timestamps = len(timeStamp_to_process_dict[timeStamps[current_timestamp_index]])
weightageList = random.sample(range(1, 100), num_timestamps)
weightSum = sum(weightageList)
total_watts = wattages[current_timestamp_index]

for i in range(num_timestamps):
  watts = float(total_watts)*weightageList[i]/weightSum
  toInsert = watts #changed
  timeStamp_to_process_dict[timeStamps[current_timestamp_index]][i].insert(1, toInsert)


In [7]:
timeStamp_to_process_dict

{'12:29:55': [['12:29:46',
   5.351848739495798,
   'exit:',
   'pid=30976',
   'exit_code=0',
   '(kworker/u16:11)'],
  ['12:29:48',
   3.6181512605042023,
   'fork:',
   'parent',
   'pid=3162',
   '->',
   'clone',
   'pid=32537',
   '(/usr/lib/tracker/tracker-miner-fs)']],
 '12:30:05': [],
 '12:30:15': [['12:30:09',
   5.254615384615385,
   'fork:',
   'parent',
   'pid=2',
   '->',
   'fork',
   'pid=32539',
   '(kworker/u16:0)'],
  ['12:30:12',
   3.715384615384616,
   'fork:',
   'parent',
   'pid=18976',
   '->',
   'clone',
   'pid=32540',
   '(/opt/google/chrome/chrome',
   '--type=utility',
   '--utility-sub-type=storage.mojom.StorageService',
   '--field-trial-handle=1537189213329473479,10046402530275150138,131072',
   '--lang=en-GB',
   '--service-sandbox-type=utility',
   '--enable-crashpad',
   '--crashpad-handler-pid=18932',
   '--enable-crash-reporter=db9b27bb-f8e0-48f2-9946-db3a1723db6c,',
   '--shared-files=v8_context_snapshot_data:100)']],
 '12:30:25': [['12:30:20',

In [9]:
consumption_pid_list = []

for List in timeStamp_to_process_dict.values():
  for  item in List:
    #modify item - append the process details together
    n = len(item)

    if (item[n-1][0] == '('):
      continue

    else:
      i = n-1

      #mash the whole process detail string together
      while (item[i][0] != '('):
        item[i-1] += " "
        item[i-1] += item[i]
        item.pop()
        i-=1
    
    #for pids
    if (len(item) == 9):
      #in this case, retain only the clone pid
      del item[3:7]
    
    elif (len(item) == 6):
      #in this case, retain the pid but not the exit code
      del item[4]

    #perhaps there is another value of length that can come up - in which case the remove "pid=" will not work
    else:
      print(len(item))

    #finally, in the pid column, remove the "pid=" part from the string
    item[3] = int(item[3][4:])

    #append it to list
    consumption_pid_list.append(item)

consumption_pid_list

[['12:31:03',
  8.95,
  'exit:',
  32543,
  '(/opt/google/chrome/chrome --type=utility --utility-sub-type=audio.mojom.AudioService --field-trial-handle=1537189213329473479,10046402530275150138,131072 --lang=en-GB --service-sandbox-type=none --enable-crashpad --crashpad-handler-pid=18932 --enable-crash-reporter=db9b27bb-f8e0-48f2-9946-db3a1723db6c, --shared-files=v8_context_snapshot_data:100 --enable-crashpad)'],
 ['12:31:12',
  1.5663333333333334,
  'exit:',
  32545,
  '(/opt/google/chrome/chrome --type=renderer --field-trial-handle=1537189213329473479,10046402530275150138,131072 --lang=en-GB --enable-crashpad --crashpad-handler-pid=18932 --enable-crash-reporter=db9b27bb-f8e0-48f2-9946-db3a1723db6c, --extension-process --num-raster-threads=4 --enable-main-frame-before-activation --renderer-client-id=89 --no-v8-untrusted-code-mitigations --shared-files=v8_context_snapshot_data:100)'],
 ['12:31:12',
  3.0903333333333336,
  'fork:',
  32549,
  '(/opt/google/chrome/chrome --type=utility --

In [10]:
consumption_pid_df = pd.DataFrame(consumption_pid_list, columns = ["Timestamp", "Watts", "Process", "PID", "Message"])
consumption_pid_df

,Timestamp,Watts,Process,PID,Message
0,12:31:03,8.950000,exit:,32543,(/opt/google/chrome/chrome --type=utility --ut...
1,12:31:12,1.566333,exit:,32545,(/opt/google/chrome/chrome --type=renderer --f...
2,12:31:12,3.090333,fork:,32549,(/opt/google/chrome/chrome --type=utility --ut...
3,12:31:15,2.159000,exit:,32546,(/opt/google/chrome/chrome --type=utility --ut...
4,12:31:45,0.270193,exit:,32549,(/opt/google/chrome/chrome --type=utility --ut...
5,12:31:50,1.391870,exit:,32551,(/opt/google/chrome/chrome --enable-crashpad)
6,12:31:53,3.618862,fork:,32562,(/opt/google/chrome/chrome --type=utility --ut...
7,12:31:53,0.765528,fork:,32563,(/opt/google/chrome/chrome --type=renderer --f...
8,12:31:53,2.783740,fork:,32564,(/opt/google/chrome/chrome --type=renderer --f...
9,12:32:03,1.737647,fork:,32566,(/opt/google/chrome/chrome --type=renderer --f...


Change the path to whatever you want it to be in your google drive - this will output the **final measurements** including process ids and process details messages

In [16]:
pid_timestamp_measurements_path = '/content/drive/MyDrive/HSBC/march-to-sustainability/pid_timestamp_measurements.csv'

with open(pid_timestamp_measurements_path, 'w', encoding = 'utf-8-sig') as f1:
  consumption_pid_df.to_csv(f1)

#DON'T RERUN THIS BLOCK AFTER FIRST TIME

In [ ]:
print(power_stats)

[['Average', '0.2', '0.0', '0.1', '98.8', '0.9', '1.0', '352.3', '123.4', '2.1', '0.1', '2.5', '7.85'], ['StdDev', '0.1', '0.1', '0.1', '0.6', '0.6', '0.0', '86.1', '26.3', '1.6', '0.7', '2.1', '1.12'], ['Minimum', '0.1', '0.0', '0.1', '97.1', '0.2', '1.0', '261.2', '84.1', '0.0', '0.0', '0.0', '5.92'], ['Maximum', '0.5', '0.3', '0.4', '99.6', '2.5', '1.0', '722.4', '232.2', '6.0', '4.0', '9.0', '8.98']]
